In [15]:
import pandas as pd
import pickle

In [16]:
%%time

individual_provider_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         dtype={'Zip Code of the Provider': object},
                         chunksize = 50000):
    individual_provider_rows.append(chunk[chunk['Entity Type of the Provider'] == 'I']) 
               
                
individual_provider_payment_df = pd.concat(individual_provider_rows, ignore_index=True)

ParserError: Error tokenizing data. C error: out of memory

In [17]:
# creating total medicare revenue column

individual_provider_payment_df['Total Medicare Revenue'] = individual_provider_payment_df['Average Medicare Payment Amount'] * individual_provider_payment_df['Number of Services']

MemoryError: Unable to allocate 71.8 MiB for an array with shape (9416125,) and data type float64

In [ ]:
# creates and adds groupings for codes to dataframe

HCPCS_Grouped_Code = []

for value in individual_provider_payment_df['HCPCS Code']:
    if value.endswith("T"):
        HCPCS_Grouped_Code.append("0000T")
    else:
        HCPCS_Grouped_Code.append(value[0:3] + "00")


individual_provider_payment_df['HCPCS Grouped Code'] = HCPCS_Grouped_Code 

In [ ]:
individual_provider_payment_df.shape

In [ ]:
individual_provider_payment_df.columns

In [ ]:
individual_provider_payment_df_clean = individual_provider_payment_df.drop(columns = ['Last Name/Organization Name of the Provider',
                                                                                      'First Name of the Provider', 
                                                                                      'Middle Initial of the Provider',
                                                                                      'Entity Type of the Provider',
                                                                                      'Street Address 1 of the Provider',
                                                                                      'Street Address 2 of the Provider',
                                                                                      'City of the Provider',
                                                                                      'Zip Code of the Provider',
                                                                                      'State Code of the Provider',
                                                                                      'Country Code of the Provider',
                                                                                      'Medicare Participation Indicator',
                                                                                      'HCPCS Drug Indicator',
                                                                                      'Average Medicare Allowed Amount', 
                                                                                      'Average Submitted Charge Amount',
                                                                                      'Average Medicare Standardized Amount', 
                                                                                      ])
                                                                           
#  Keeping 'National Provider Identifier', 
#          'Credentials of the Provider',
#          'Gender of the Provider', 
#         'Provider Type', 
#         'Place of Service', 
#         'HCPCS Code',
#         'HCPCS Description', 
#         'HCPCS Grouped Code',
#         'Total Medicare Revenue', 
#         'Average Medicare Payment Amount',
#         'Number of Services',
#         'Number of Medicare Beneficiaries',
#         'Number of Distinct Medicare Beneficiary/Per Day Services'


# new code to add ratios and calculations

In [ ]:
# creates sums over Provider Type and Provider Type/HCPCS Code

provider_type_sum_of_services = individual_provider_payment_df_clean.groupby('Provider Type', as_index=False)['Number of Services'].sum()

provider_type_services = individual_provider_payment_df_clean.groupby(['Provider Type','HCPCS Code'], as_index=False)['Number of Services'].sum()

provider_type_grouped_services = individual_provider_payment_df_clean.groupby(['Provider Type','HCPCS Grouped Code'], as_index=False)['Number of Services'].sum()

In [ ]:
# creates data frames from the new sums

provider_type_sum_of_services_df = pd.DataFrame({"Provider Type": provider_type_sum_of_services['Provider Type'], "total_count": provider_type_sum_of_services["Number of Services"]})

provider_type_services_df = pd.DataFrame({ "Provider Type":provider_type_services["Provider Type"], "HCPCS Code":provider_type_services["HCPCS Code"], "service_count":provider_type_services["Number of Services"]})

provider_type_grouped_services_df = pd.DataFrame({"Provider Type": provider_type_grouped_services['Provider Type'], "HCPCS Grouped Code":provider_type_grouped_services["HCPCS Grouped Code"], "grouped_service_count":provider_type_grouped_services["Number of Services"]})


In [ ]:
individual_provider_payment_df_clean_merged = pd.merge(individual_provider_payment_df_clean, provider_type_services_df, how="left", on=['Provider Type','HCPCS Code'])
individual_provider_payment_df_clean_merged = pd.merge(individual_provider_payment_df_clean_merged, provider_type_sum_of_services_df, how="left", on='Provider Type')
individual_provider_payment_df_clean_merged = pd.merge(individual_provider_payment_df_clean_merged, provider_type_grouped_services_df, how="left", on=['Provider Type','HCPCS Grouped Code'])
# individual_provider_payment_df_clean_merged['service_ratio'] = individual_provider_payment_df_clean_merged.service_count/individual_provider_payment_df_clean_merged.total_count
# individual_provider_payment_df_clean_merged['grouped_service_ratio'] = individual_provider_payment_df_clean_merged.grouped_service_count/individual_provider_payment_df_clean_merged.total_count

individual_provider_payment_df_clean_merged.info()

In [ ]:
individual_provider_payment_df_clean_merged.sort_values(by = 'HCPCS Code').head()

In [ ]:
individual_provider_payment_df_clean_merged.to_csv("../data/final_medicare_provider_file.csv", index = False)


In [ ]:
individual_provider_payment_df_clean_merged['HCPCS Code'].nunique()

In [ ]:
# optional pickle

# individual_provider_payment_df_clean.to_pickle("../data/individual_provider_payment.pkl")